# Introduction to Geospatial Data Analysis with GDAL

## Reference
GDAL Class
https://gdal.org/python/osgeo.ogr-module.html

# Ex.1: Create New Shapefile
## Warning Case : Thai Language Codepage

In [ ]:
#-*-coding:UTF-8-*-

import gdal, ogr, osr
import os

#(1) Define Datasource (Shapefile filename)
gdal.SetConfigOption('SHAPE_ENCODING', '') #Always Use
driver = ogr.GetDriverByName('ESRI Shapefile')
layername = 'New_Point'
output_path = 'D:\\Conversion_Area\\Python_Training\\Data\\'
output_fn = output_path + layername + '.shp'
#Check Existing Shapefile 
if os.path.exists(output_fn):
    driver.DeleteDataSource(output_fn)
datasource = driver.CreateDataSource(output_fn)
    
#(2) Define Projection
Prj = osr.SpatialReference()
Prj.ImportFromEPSG(32647)         #UTM WGS84 Zone 47
#Prj.ImportFromEPSG(4326)         #Lat/Long WGS84

#(3) Create Layer
layer = datasource.CreateLayer(layername, srs=Prj, geom_type=ogr.wkbPoint, options=['ENCODING=LDID/0'])
#layer = datasource.CreateLayer(layername, srs=Prj, geom_type=ogr.wkbPoint) #Not Use

#(4) Define Encoding & Create CPG file
outcpgfile = output_fn.replace('.shp','.cpg')
cpgfile = open(outcpgfile,'w')
#cpgfile.write('UTF-8')
cpgfile.write('OEM 874')
cpgfile.close()
    
#(5) Define Fields
fldDef = ogr.FieldDefn('ID', ogr.OFTInteger)
fldDef.SetWidth(10)
layer.CreateField(fldDef)
fldDef.Destroy()

fldDef = ogr.FieldDefn('NAME', ogr.OFTString)
fldDef.SetWidth(100)
layer.CreateField(fldDef)
fldDef.Destroy()

fldDef = ogr.FieldDefn('NAME_ENG', ogr.OFTString)
fldDef.SetWidth(100)
layer.CreateField(fldDef)
fldDef.Destroy()

fldDef = ogr.FieldDefn('X', ogr.OFTReal)
fldDef.SetWidth(13)
fldDef.SetPrecision(3)
layer.CreateField(fldDef)
fldDef.Destroy()

fldDef = ogr.FieldDefn('Y', ogr.OFTReal)
fldDef.SetWidth(13)
fldDef.SetPrecision(3)
layer.CreateField(fldDef)
fldDef.Destroy()

#(6) Create Feature(Record)
#Merkator@ตึกไทย
X = 668334.249
Y = 1517293.049
feature = ogr.Feature(layer.GetLayerDefn())
pt = ogr.Geometry(ogr.wkbPoint)
pt.AddPoint(X, Y)
feature.SetGeometry(pt)
feature.SetField('ID', 1)
#feature.SetField('NAME', 'บริษัทเมอร์เคเทอร์ จำกัด'.decode('UTF8').encode('CP874'))
feature.SetField('NAME', u'บริษัทเมอร์เคเทอร์ จำกัด'.encode('CP874'))
#feature.SetField('NAME', 'บริษัทเมอร์เคเทอร์ จำกัด')
feature.SetField('NAME_ENG', 'Merkator Co., Ltd.')
feature.SetField('X', X)
feature.SetField('Y', Y)
layer.CreateFeature(feature)
pt.Destroy()
feature.Destroy()

datasource.Destroy()
print 'Create Shapefile Complete!'

# Ex.2: Create Centroid from Polygon Layer

In [ ]:
#-*-coding:UTF-8-*-

import gdal, ogr, osr
import os

## Create Shapefile Module

In [ ]:
def CreateShapefile(output_path, layername, Prj, ShapeType):
    #(1) Define Datasource (Shapefile filename)
    gdal.SetConfigOption('SHAPE_ENCODING', '')
    driver = ogr.GetDriverByName('ESRI Shapefile')
    output_fn = output_path + layername + '.shp'
    #Check Existing Shapefile 
    if os.path.exists(output_fn):
        driver.DeleteDataSource(output_fn)
    datasource = driver.CreateDataSource(output_fn)

    #(2) Create Layer
    layer = datasource.CreateLayer(layername, srs=Prj, geom_type=ShapeType, options=['ENCODING=LDID/0'])

    #(3) Define Encoding & Create CPG file
    outcpgfile = output_fn.replace('.shp','.cpg')
    cpgfile = open(outcpgfile,'w')
    cpgfile.write('OEM 874')
    cpgfile.close()
    
    #(4) Define Fields
    fldDef = ogr.FieldDefn('ID', ogr.OFTInteger)
    fldDef.SetWidth(10)
    layer.CreateField(fldDef)
    fldDef.Destroy()

    fldDef = ogr.FieldDefn('NAME1', ogr.OFTString)
    fldDef.SetWidth(100)
    layer.CreateField(fldDef)
    fldDef.Destroy()
    
    fldDef = ogr.FieldDefn('NAME2', ogr.OFTString)
    fldDef.SetWidth(100)
    layer.CreateField(fldDef)
    fldDef.Destroy()
    
    fldDef = ogr.FieldDefn('NAME3', ogr.OFTString)
    fldDef.SetWidth(100)
    layer.CreateField(fldDef)
    fldDef.Destroy()
        
    return datasource, layer

## Main Module

In [ ]:
#(1) Assign Admin_Poly Layer (Polygon Feature) <Input Layer>
driver = ogr.GetDriverByName('ESRI Shapefile')
working_path = 'D:\\Conversion_Area\\Python_Training\\Data\\'
datasource = driver.Open(working_path+ 'Admin_BKK.shp', 0)
layer = datasource.GetLayer()
Source_SRF =  layer.GetSpatialRef()

#(2) Assign Centroid Shapefile (Point Feature) <Output Layer>
layername = 'Admin_Centroid'
ds, OutputLYR = CreateShapefile(working_path, layername, Source_SRF, ogr.wkbPoint)
    
#(3) Loop Input Layer
recset = layer.GetNextFeature()
record = 0
while recset:
    geom = recset.GetGeometryRef()
    record +=1    
    #(4) Create Feature(Record)
    feature = ogr.Feature(OutputLYR.GetLayerDefn())
    Centroid = geom.Centroid()
    feature.SetGeometry(Centroid)
    feature.SetField('ID', record)
    feature.SetField('NAME1', recset.GetFieldAsString('NAME1').decode('CP874').encode('CP874'))
    feature.SetField('NAME2', recset.GetFieldAsString('NAME2').decode('CP874').encode('CP874'))
    feature.SetField('NAME3', recset.GetFieldAsString('NAME3').decode('CP874').encode('CP874'))
    OutputLYR.CreateFeature(feature)
    feature.Destroy()

    recset.Destroy()
    recset = layer.GetNextFeature()

datasource.Destroy()
ds.Destroy()

print 'Generate Centroid Complete!'

# Ex.3: Add Fields & Update Attribute in Existing Shapefile

In [ ]:
#-*-coding:UTF-8-*-

import gdal, ogr, osr
import os

#(1) Assign Existing Layer
driver = ogr.GetDriverByName('ESRI Shapefile')
working_path = 'D:\\Conversion_Area\\Python_Training\\Data\\'
datasource = driver.Open(working_path + 'L_trans_sample.shp', 1)
layer = datasource.GetLayer()
    
#(2) Add New Fields
fldDef = ogr.FieldDefn('MINUTES', ogr.OFTReal)
fldDef.SetWidth(9)
fldDef.SetPrecision(3)
layer.CreateField(fldDef)
fldDef.Destroy()

#(3) Loop on Exiting Shapefile
record = 0
recset = layer.GetNextFeature()
while recset:
    record += 1
    geom = recset.GetGeometryRef()
    AVG_Speed = recset.GetField('AVG_SPEED')
    Meters = recset.GetField('LENGTH')
    Minutes = (Meters * 60) / (AVG_Speed * 1000.0)
    
    #(4) Update Fields    
    recset.SetField('MINUTES', Minutes)
    layer.SetFeature(recset)

    recset.Destroy()
    recset = layer.GetNextFeature()

datasource.Destroy()

print 'Update Field Complete!'

# Ex.4: Change Fields Name/ Delete Fields in Existing Shapefile

In [ ]:
#-*-coding:UTF-8-*-

import gdal, ogr, osr
import os

#(1) Assign Existing Layer
driver = ogr.GetDriverByName('ESRI Shapefile')
working_path = 'D:\\Conversion_Area\\Python_Training\\Data\\'
datasource = driver.Open(working_path + 'Landmark_sample_bak.shp', 1)
layer = datasource.GetLayer()

#(2) Change Fields Name (Mu --> Village_No)
n = layer.GetLayerDefn().GetFieldIndex('MU')
fldDef = ogr.FieldDefn('VILLAGE_NO', ogr.OFTString)
layer.AlterFieldDefn(n, fldDef, ogr.ALTER_NAME_FLAG)

#(3) Change Fields Width/Precision (Region[String20] --> Region[String50])
n = layer.GetLayerDefn().GetFieldIndex('REGION')
fldDef.SetWidth(50)
layer.AlterFieldDefn(n, fldDef, ogr.ALTER_WIDTH_PRECISION_FLAG)

#(4) Change Fields Type
n = layer.GetLayerDefn().GetFieldIndex('CITY_CODE')
fldDef = ogr.FieldDefn('CITY_CODE', ogr.OFTString)
layer.AlterFieldDefn(n, fldDef, ogr.ALTER_TYPE_FLAG)

#(5) Delete Fields
layer.DeleteField(layer.GetLayerDefn().GetFieldIndex('LO_CODE_2'))
layer.DeleteField(layer.GetLayerDefn().GetFieldIndex('LO_CODE_3'))
layer.DeleteField(layer.GetLayerDefn().GetFieldIndex('LO_CODE_4'))
layer.DeleteField(layer.GetLayerDefn().GetFieldIndex('LO_CODE_5'))
    
datasource.Destroy()

print 'Change Field Complete!'

# Ex.5: Export to New Shapefile Base on Attribute

In [ ]:
#-*-coding:UTF-8-*-

import gdal, ogr, osr
import os

#(1) Assign Existing Layer
gdal.SetConfigOption('SHAPE_ENCODING', '')
driver = ogr.GetDriverByName('ESRI Shapefile')
working_path = 'D:\\Conversion_Area\\Python_Training\\Data\\'
datasource = driver.Open(working_path + 'Landmark_sample.shp', 0)
layer = datasource.GetLayer()

#(2) Select Attribute Filter (ATM)
layer.SetAttributeFilter("POI_CODE = 9102")

#(3) Create Output Datasource
out_fn = working_path + 'ATM.shp'
out_datasource = driver.CreateDataSource(out_fn)
out_layer = out_datasource.CopyLayer(layer, out_fn, options=['ENCODING=LDID/0'])

#(4) Define Encoding & Create CPG file
outcpgfile = out_fn.replace('.shp','.cpg')
cpgfile = open(outcpgfile,'w')
cpgfile.write('OEM 874')
cpgfile.close()

datasource.Destroy()
out_datasource.Destroy()

print 'Export Shapefile Complete!'